In [4]:
import cv2#imports openCV into notebook
import mediapipe as mp#imports MediaPipe
import numpy as np#imports Numpy
mp_drawing = mp.solutions.drawing_utils#Gives the drawing utilities
mp_pose = mp.solutions.pose#imports pose estimation models from mediapipe from a range of models for face and other parts

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [7]:
def calculate_angle(a,b,c):
    a=np.array(a) #First
    b=np.array(b) #Mid
    c=np.array(c) #End
    #makes it easier to calculate angles and make it numpy arrays
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0])-np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    #Calculates the radians for a particular angle
    
    if(angle>180.0):
        angle =360-angle
    #convert angle between zero and 180
        
    return angle

In [20]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Define connections for the robot-mirroring arm (shoulder-elbow, elbow-wrist)
CONNECTIONS = [
    (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW),
    (mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST),
    (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP),
    (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),  # Across shoulders
    (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP),  # Across hips
    (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP),
]


cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor the image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract the landmarks
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # Draw specific landmarks and connections
            for connection in CONNECTIONS:
                start = connection[0].value
                end = connection[1].value
                # Get the start and end landmark positions
                start_landmark = landmarks[start]
                end_landmark = landmarks[end]
                
                # Draw circle for the landmarks
                cv2.circle(image, (int(start_landmark.x * 640), int(start_landmark.y * 480)), 5, (245, 117, 66), -1)
                cv2.circle(image, (int(end_landmark.x * 640), int(end_landmark.y * 480)), 5, (245, 117, 66), -1)
                
                # Draw line for the connection
                cv2.line(image,
                         (int(start_landmark.x * 640), int(start_landmark.y * 480)),
                         (int(end_landmark.x * 640), int(end_landmark.y * 480)),
                         (245, 66, 230), 2)
            
            # Calculate and display the angle
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            angle = calculate_angle(shoulder, elbow, wrist)
            cv2.putText(image, str(angle),
                        tuple(np.multiply(elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()